In [115]:
import numpy as np
import pandas as pd
import seaborn as sns
import csv
import re
import math as m

In [2]:
durham_data = pd.read_stata("fips_37063.dta")
raleigh_data = pd.read_stata("fips_37183.dta")

In [3]:
for c in durham_data.columns:
    print(str(c) + ": " + str(durham_data[c].dtype))

fips_code: int32
apn_number_unformatted: int32
apn_sequence_number: int8
apn_number_formatted: int32
original_apn: int32
previous_parcel_number: float64
pcl_id_iris_formatted: int32
account_number: float64
map_reference_1: object
map_reference_2: object
census_tract: float64
block_number: object
lot_number: object
range: float64
township: float64
section: float64
quarter_section: float64
flood_zone_community_panel_id: int32
land_use_code: int16
county_land_use_1: float64
county_land_use_2: float64
mobile_home_indicator_flag: object
zone_code: object
property_indicator_code: int8
subdivision_tract_number: object
subdivision_plat_book: object
subdivision_plat_page: object
subdivision_name: object
blocklevellatitude: float64
blocklevellongitude: object
property_house_number_prefix: int8
property_house_number: int16
property_house_number_suffix: object
property_direction: object
property_street_name: object
property_mode: object
property_quadrant: object
property_apartment_unit_number: obj

In [4]:
property_list = []
for c in durham_data.columns:
    if 'property' in c:
        property_list.append(c)
durham_data.loc[:, property_list]

,property_indicator_code,property_house_number_prefix,property_house_number,property_house_number_suffix,property_direction,property_street_name,property_mode,property_quadrant,property_apartment_unit_number,property_city,property_state,property_zipcode,property_carrier_route
0,21,0,918,,,LANCASTER,ST,,,DURHAM,NC,27701.0,C009
1,21,0,914,,,LANCASTER,ST,,,DURHAM,NC,27701.0,C009
2,21,0,910,,,LANCASTER,ST,,,DURHAM,NC,27701.0,C009
3,10,0,908,,,LANCASTER,ST,,,DURHAM,NC,27701.0,C009
4,21,0,906,,,LANCASTER,ST,,,DURHAM,NC,27701.0,C009
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68096,80,0,1244,,,EXCHANGE,PL,,,DURHAM,NC,27713.0,C010
68097,10,0,303,,,OLDCASTLE,DR,,,MORRISVILLE,NC,27560.0,R008
68098,10,0,305,,,OLDCASTLE,DR,,,MORRISVILLE,NC,27560.0,R008
68099,10,0,307,,,OLDCASTLE,DR,,,MORRISVILLE,NC,27560.0,R008


In [5]:
durham_data['address']

0        918 LANCASTER ST
1        914 LANCASTER ST
2        910 LANCASTER ST
3        908 LANCASTER ST
4        906 LANCASTER ST
               ...       
68096    1244 EXCHANGE PL
68097    303 OLDCASTLE DR
68098    305 OLDCASTLE DR
68099    307 OLDCASTLE DR
68100    309 OLDCASTLE DR
Name: address, Length: 68101, dtype: object

In [6]:
durham_data[durham_data['address'] == "306 S DRIVER ST"]

,fips_code,apn_number_unformatted,apn_sequence_number,apn_number_formatted,original_apn,previous_parcel_number,pcl_id_iris_formatted,account_number,map_reference_1,map_reference_2,...,electric_energy_code,fuel_code,sewer_code,utilities,water_type_code,address,sale_date,sale_month,sale_quarter,sale_year
7998,37063,112482,1,112482,112482,NaN,112482,NaN,0831,,...,1.0,NaN,SPU,.,1.0,306 S DRIVER ST,1994-12-27,1994-12-01,1994-10-01,1994.0


In [7]:
raleigh_data[raleigh_data['address'] == '18 W COLONY PL']

,fips_code,apn_number_unformatted,apn_sequence_number,apn_number_formatted,original_apn,previous_parcel_number,pcl_id_iris_formatted,account_number,map_reference_1,map_reference_2,...,electric_energy_code,fuel_code,sewer_code,utilities,water_type_code,address,sale_date,sale_month,sale_quarter,sale_year


In [8]:
with open("Assessor0/UNIV_ASSESSOR_0.TXT", "r") as assessor_file, open("Assessor0/new_assessor0.txt", "w") as clean_file:
    for line in assessor_file:
        if re.search(r"NC[0-9]{5}", line):
            if re.search(r"RALEIGH[\s]+NC[0-9]{5}", line) or re.search(r"DURHAM[\s]+NC[0-9]{5}", line):
                clean_file.write(line)

with open("Assessor0/new_assessor0.txt", "r") as assessor_file, open("Assessor0/assessor_wake_and_durham.txt", "w") as clean_file:
    for line in assessor_file:
        words = line.split()
        if words[3]+words[4] == "37063":
            clean_file.write(line)

with open("Assessor0/new_assessor0.txt", "r") as assessor_file, open("Assessor0/assessor_wake_and_durham.txt", "a") as clean_file:
    for line in assessor_file:
        words = line.split()
        if words[3]+words[4] == "37183":
            clean_file.write(line)

assessor_file.close()
clean_file.close()



In [193]:
cleaned_dataset = pd.read_csv('Assessor0/wake_and_durham_assessor0.csv', header=None, dtype=
{
    5: str,
    8: float, 
    22: object,
    23: str,
    24: str,
    25: str,
    26: str,
    27: str,
    28: str,
    49: float,
    50: float,
    52: float,
    53: float
}
)

/var/folders/k1/b11hy6gj2wdgdc768yvgls140000gn/T/ipykernel_20332/3647906565.py:1: DtypeWarning: Columns (75,81,104,119) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_dataset = pd.read_csv('Assessor0/wake_and_durham_assessor0.csv', header=None, dtype=


In [194]:
dropset = []

for ind, row in cleaned_dataset.iterrows():
    s = str(row[22])
    if m.isnan(row[8]):
        dropset.append(ind)
    elif ("A" in s) or ("B" in s) or ("C" in s):
        dropset.append(ind)
    else:
        f = float(row[22])
        if m.isnan(f):
            dropset.append(ind)
        elif m.isnan(row[52]):
            dropset.append(ind)
        elif m.isnan(row[53]):
            dropset.append(ind)

cleaned_dataset = cleaned_dataset.drop(dropset).reset_index().drop(['index'], axis = 1)

cleaned_dataset[8] = cleaned_dataset[8].astype(int)
cleaned_dataset[22] = cleaned_dataset[22].astype(int)
cleaned_dataset[52] = cleaned_dataset[52].astype(int)
cleaned_dataset[53] = cleaned_dataset[53].astype(int)

In [195]:
pd.set_option('display.max_columns', None)
cleaned_dataset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,77173200,437,NCDURHAM,37,63,DURHAM,1.371640e+05,8.11E+13,2007,0,DSM LIMITED PARTNERSHIP,DSM LIMITED PARTNERSHIP,NaN,NaN,NaN,NaN,1,GUNDERSDORFF & CO ACCTS,GUNDERSDORFF & CO ACCTS,NaN,NaN,1 Y,18,W,COLONY,PL,NaN,NaN,DURHAM,NC27705,5582,C00418,W,COLONY,PL,STE,210,DURHAM,NC27705,5582,C004Y,COLONY RD OFC PRK CONDOS/BLDG 300/UT#230,NaN,10,COLONY ROAD OFC PARK BL 300,0.0,COFF466,PDR-9.4,201253942,0.0,53942.0,100.00,2012,2012,53942.0,0.0,53942.0,100.0,53942.0,0.0,53942.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53942.0,2011703.08,0.0,2.010000e+16,0.0,9.0,9420,0.0,0.0,0.0,0,Y,0,0.0,0.0,0.0,9420.0,NaN,NaN,NaN,0.0,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,NaN,9420,9420.0,1.0,691,1986.0,NaN,0,NaN,0.0,NaN,NaN,NaN,138320
1,77173210,437,NCDURHAM,37,63,DURHAM,1.691410e+05,84403304088,2007,0,"AMBROSE,MARGARET B",MARGARET,B,AMBROSE,NaN,NaN,0,"PHILIP,NOAH BROWN",NOAH,BROWN,PHILIP,0TC,1830,NaN,HAMLIN,RD,NaN,NaN,DURHAM,NC27704,9650,R0052408,NaN,PAR,PL,NaN,NaN,DURHAM,NC27705,3162,C022N,BRAGTOWN HEIGHTS/LT#33,33,30,BRAGTOWN HEIGHTS,0.0,VRES311,RD,201214336,14336.0,0.0,0.00,2012,2012,14336.0,14336.0,0.0,NaN,14336.0,14336.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14336.0,2011123.07,0.0,2.010000e+16,0.0,NaN,0,0.0,0.0,0.0,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,38289.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0.0,0.0,0,NaN,NaN,0,NaN,0.0,NaN,20091217.0,63850809,138320
2,77177150,437,NCDURHAM,37,63,DURHAM,1.041420e+05,82213034244,2007,0,"OBRIANT,HOWARD B & JEAN B",HOWARD,B,OBRIANT,JEAN,B,0,NaN,NaN,NaN,NaN,HW,2707,NaN,HILLSBOROUGH,RD,NaN,NaN,DURHAM,NC27705,4043,C0023109,NaN,KENAN,RD,NaN,NaN,DURHAM,NC27704,2021,C003N,PROP-JONES R M ESTATE/LT#8,8,3,R M JONES ESTATE PROP,0.0,VCOM340,GC,201254692,54692.0,0.0,0.00,2012,2012,54692.0,54692.0,0.0,NaN,54692.0,54692.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54692.0,2011712.86,0.0,2.010000e+16,0.0,NaN,0,0.0,0.0,0.0,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,0.0,7449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,NaN,0,0.0,0.0,0,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,138320
3,77177160,437,NCDURHAM,37,63,DURHAM,1.040850e+05,82217027181,2007,0,"EDWARDS,BEULAH MAE",BEULAH,MAE,EDWARDS,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2608,W,MAIN,ST,NaN,NaN,DURHAM,NC27705,3912,C0022608,W,MAIN,ST,NaN,NaN,DURHAM,NC27705,3912,C002Y,ERWIN COTTON MLS/SEC:01/BLK:D/LT#05,5,D,ERWIN COTTON MILLS SEC 01,0.0,RSFR111,R-3,201285435,15358.0,70077.0,82.02,2012,2012,85435.0,15358.0,70077.0,82.0,85435.0,15358.0,70077.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85435.0,2011556.77,0.0,2.010000e+16,0.0,9.0,1357,0.0,0.0,0.0,0,Y,1357,0.0,0.0,0.0,1357.0,NaN,G,0Y,0.0,2,0.0,0.0,8494.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1,1.0,2.0,1357,1357.0,1.0,1 1,1909.0,NaN,0,NaN,0.0,NaN,20020101.0,258000297,138320
4,77177170,437,NCDURHAM,37,63,DURHAM,1.040950e+05,82213036038,2007,0,"MILLAR,KIM K",KIM,K,MILLAR,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2625,NaN,HILLSBOROUGH,RD,NaN,NaN,DURHAM,NC27705,4080,C002704,NaN,FLAGSTONE,WAY,NaN,NaN,DURHAM,NC27712,9515,R017N,PROP-CRABTREE A J/LT#03 GR PT,3,3,NaN,0.0,MTRA420,GC,2012249907,64761.0,185146.0,74.08,2012,2012,249907.0,64761.0,185146.0,74.0,249907.0,64761.0,185146.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249907.0,20113257.28,0.0,2.010000e+16,0.0,8.0,2809,0.0,0.0,0.0,0,Y,2809,0.0,0.0,0.0,2809.0,NaN,NaN,NaN,0.0,Y,0.0,0.0,8015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,NaN,2809,2809.0,1.0,1,1980.0,1980,0,NaN,0.0,NaN,NaN,NaN,138320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [188]:
state = []
zips = []
for s in cleaned_dataset[29]:
    state.append(s[0:2])
    zips.append(s[2:])
df1 = pd.DataFrame({"property_state": state, "property_zipcode": zips})

state2 = []
zips2 = []
for s in cleaned_dataset[38]:
    state2.append(s[0:2])
    zips2.append(s[2:])
df2 = pd.DataFrame({'a': state2, 'b': zips2})

print(df2)

cleaned_dataset = pd.concat([cleaned_dataset.iloc[:, :29], df1, cleaned_dataset.iloc[:, 30:38], df2, cleaned_dataset.iloc[:,39:]], axis = 1)

        a      b
0      NC  27705
1      NC  27705
2      NC  27704
3      NC  27705
4      NC  27712
...    ..    ...
28558  NC  27607
28559  NC  27607
28560  NC  27603
28561  NC  27603
28562  NC  27603

[28563 rows x 2 columns]


In [196]:
cleaned_dataset = cleaned_dataset.drop(list(range(0,5)) + list(range(6,8)) + list(range(9,22)), axis = 1).rename(columns={5: "county", 8: "assessed_year", 22: "property_house_number", 23: 'property_direction', 24: 'property_street_name', 25: 'property_mode', 26: 'property_quadrant', 27: 'property_apartment_unit_number', 28: 'property_city', 49: "tax_assessed_value1?", 50: "tax_assessed_value2?", 52:"build_year?", 53: "remodel_year?"}).replace(np.NaN, "")

cleaned_dataset

,county,assessed_year,property_house_number,property_direction,property_street_name,property_mode,property_quadrant,property_apartment_unit_number,property_city,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,tax_assessed_value1?,tax_assessed_value2?,51,build_year?,remodel_year?,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119
0,DURHAM,2007,18,W,COLONY,PL,,,DURHAM,NC27705,5582,C00418,W,COLONY,PL,STE,210,DURHAM,NC27705,5582,C004Y,COLONY RD OFC PRK CONDOS/BLDG 300/UT#230,,10,COLONY ROAD OFC PARK BL 300,0.0,COFF466,PDR-9.4,201253942,0.0,53942.0,100.00,2012,2012,53942.0,0.0,53942.0,100.0,53942.0,0.0,53942.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53942.0,2011703.08,0.0,2.010000e+16,0.0,9.0,9420,0.0,0.0,0.0,0,Y,0,0.0,0.0,0.0,9420.0,,,,0.0,Y,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,,9420,9420.0,1.0,691,1986.0,,0,,0.0,,,,138320
1,DURHAM,2007,1830,,HAMLIN,RD,,,DURHAM,NC27704,9650,R0052408,,PAR,PL,,,DURHAM,NC27705,3162,C022N,BRAGTOWN HEIGHTS/LT#33,33,30,BRAGTOWN HEIGHTS,0.0,VRES311,RD,201214336,14336.0,0.0,0.00,2012,2012,14336.0,14336.0,0.0,,14336.0,14336.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14336.0,2011123.07,0.0,2.010000e+16,0.0,,0,0.0,0.0,0.0,0,,0,0.0,0.0,0.0,0.0,,,,0.0,,0.0,0.0,38289.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,,0,0.0,0.0,0,,,0,,0.0,,20091217.0,63850809,138320
2,DURHAM,2007,2707,,HILLSBOROUGH,RD,,,DURHAM,NC27705,4043,C0023109,,KENAN,RD,,,DURHAM,NC27704,2021,C003N,PROP-JONES R M ESTATE/LT#8,8,3,R M JONES ESTATE PROP,0.0,VCOM340,GC,201254692,54692.0,0.0,0.00,2012,2012,54692.0,54692.0,0.0,,54692.0,54692.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54692.0,2011712.86,0.0,2.010000e+16,0.0,,0,0.0,0.0,0.0,0,,0,0.0,0.0,0.0,0.0,,,,0.0,,0.0,0.0,7449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,,0,0.0,0.0,0,,,0,,0.0,,,,138320
3,DURHAM,2007,2608,W,MAIN,ST,,,DURHAM,NC27705,3912,C0022608,W,MAIN,ST,,,DURHAM,NC27705,3912,C002Y,ERWIN COTTON MLS/SEC:01/BLK:D/LT#05,5,D,ERWIN COTTON MILLS SEC 01,0.0,RSFR111,R-3,201285435,15358.0,70077.0,82.02,2012,2012,85435.0,15358.0,70077.0,82.0,85435.0,15358.0,70077.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85435.0,2011556.77,0.0,2.010000e+16,0.0,9.0,1357,0.0,0.0,0.0,0,Y,1357,0.0,0.0,0.0,1357.0,,G,0Y,0.0,2,0.0,0.0,8494.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1,1.0,2.0,1357,1357.0,1.0,1 1,1909.0,,0,,0.0,,20020101.0,258000297,138320
4,DURHAM,2007,2625,,HILLSBOROUGH,RD,,,DURHAM,NC27705,4080,C002704,,FLAGSTONE,WAY,,,DURHAM,NC27712,9515,R017N,PROP-CRABTREE A J/LT#03 GR PT,3,3,,0.0,MTRA420,GC,2012249907,64761.0,185146.0,74.08,2012,2012,249907.0,64761.0,185146.0,74.0,249907.0,64761.0,185146.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,249907.0,20113257.28,0.0,2.010000e+16,0.0,8.0,2809,0.0,0.0,0.0,0,Y,2809,0.0,0.0,0.0,2809.0,,,,0.0,Y,0.0,0.0,8015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,,2809,2809.0,1.0,1,1980.0,1980,0,,0.0,,,,138320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28558,WAKE,2010,404,,LONE PINE,LOOP,,,,NC00000,0,1210,,TRINITY,RD,#,102,RALEIGH,NC27607,4945,C025N,OPEN SPACE PARKER POINTE PH1A BM2009-01277,,12,,0.0,VMSC00,R8,2011520,520.0,0.0,0.00,2011,2011,0.0,0.0,0.0,,520.0,520.0,0.0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,380.0,20114.78,0.0,2.010000e+16,0.0,,0.0,0.0,0.0,0.0,0,,0,0.0,0.0,0.0,0.0,,,,0.0,,0.0,0.0,5663.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,,0,0.0,0.0,0,,,0,,0.0,,,,137818
28559,WAKE,2010,807,,SEQUOIA RIDGE,DR,,,FUQUAY VARINA,NC27526,4985,C0011210,,TRINITY,RD,#,102,RALEIGH,NC27607,4945,C025N,OPEN SPACE PARKER POINTE PH1A BM2009-01276,,12,,0.0,VMSC00,R8,201

In [197]:
cleaned_dataset.to_csv('Assessor0/cleaned_assessor.csv')